In [2]:
import pandas as pd
from pathlib import Path
import sys
import yfinance as yf

sys.path.append("../../")

from black_scholes import *

For this project we have data from OptionsDX.
Here is a preview:

In [4]:
try:
    df = pd.read_csv("2020/spx_eod_202001.txt")
except FileNotFoundError:
    print(
        """
        Error: missing data. The data .txt files are not included in the github repository due to repository size restrictions.
        The output required from running this file (processing.ipynb) should already exist in the /data directory.
        This file should no longer be necessary after the project in complete.
        If you have to run this notebook for some reason, download SPX EOD options data for the years specified below from
        https://www.optionsdx.com/product/spx-option-chain/.
        Then extract the zipped files and create a directory structure that looks like this:

        implied-stock-distributions/
        ├─ data/
        │  ├─ raw/
        │  │  ├─ 2020/
        │  │  │  ├─ spx_eod_202001.txt
        │  │  │  ├─ spx_eod_202002.txt
        │  │  │  └─ etc.
        │  │  ├─ 2021/
        │  │  │  ├─ spx_eod_202101.txt
        │  │  │  ├─ spx_eod_202102.txt
        │  │  │  └─ etc.
        │  │  └─ processing.ipynb
        │  ├─ spx_options_2020.csv
        │  └─ spx_options_2021.csv
        └─ project.ipynb
        """
    )
display(df)

,[QUOTE_UNIXTIME],[QUOTE_READTIME],[QUOTE_DATE],[QUOTE_TIME_HOURS],[UNDERLYING_LAST],[EXPIRE_DATE],[EXPIRE_UNIX],[DTE],[C_DELTA],[C_GAMMA],...,[P_LAST],[P_DELTA],[P_GAMMA],[P_VEGA],[P_THETA],[P_RHO],[P_IV],[P_VOLUME],[STRIKE_DISTANCE],[STRIKE_DISTANCE_PCT]
0,1577998800,2020-01-02 16:00,2020-01-02,16.0,3258.14,2020-01-03,1578085200,1.0,1.00000,0.00000,...,0.00,-0.00044,0.0,0.00100,-0.02473,0.00000,3.226840,,1858.1,0.570
1,1577998800,2020-01-02 16:00,2020-01-02,16.0,3258.14,2020-01-03,1578085200,1.0,1.00000,0.00000,...,0.05,0.00000,0.0,0.00155,-0.02480,0.00000,2.972420,20.000000,1758.1,0.540
2,1577998800,2020-01-02 16:00,2020-01-02,16.0,3258.14,2020-01-03,1578085200,1.0,1.00000,0.00000,...,0.00,-0.00032,0.0,0.00148,-0.02478,0.00000,2.733440,,1658.1,0.509
3,1577998800,2020-01-02 16:00,2020-01-02,16.0,3258.14,2020-01-03,1578085200,1.0,1.00000,0.00000,...,0.05,-0.00063,0.0,0.00145,-0.02458,-0.00016,2.510470,16.000000,1558.1,0.478
4,1577998800,2020-01-02 16:00,2020-01-02,16.0,3258.14,2020-01-03,1578085200,1.0,1.00000,0.00000,...,0.00,-0.00036,0.0,0.00115,-0.02531,0.00000,2.300260,,1458.1,0.448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141022,1580504400,2020-01-31 16:00,2020-01-31,16.0,3224.93,2022-12-16,1671224400,1050.0,0.06871,0.00013,...,0.00,-1.00000,0.0,-11760.69979,0.00000,0.00000,,,1175.1,0.364
141023,1580504400,2020-01-31 16:00,2020-01-31,16.0,3224.93,2022-12-16,1671224400,1050.0,0.05396,0.00019,...,0.00,-1.00000,0.0,-12760.70021,0.00000,0.00000,,,1275.1,0.395
141024,1580504400,2020-01-31 16:00,2020-01-31,16.0,3224.93,2022-12-16,1671224400,1050.0,0.04295,0.00015,...,0.00,-1.00000,0.0,-13760.70017,0.00000,0.00000,,,1375.1,0.426
141025,1580504400,2020-01-31 16:00,2020-01-31,16.0,3224.93,2022-12-16,1671224400,1050.0,0.03415,0.00009,...,0.00,-1.00000,0.0,-14760.70006,0.00000,0.00000,,,1475.1,0.457


Below we'll do some cleaning and augment the dataset.
We'll make the following modifications:
- Remove columns that we don't need.
- Rename columns for convenience and readability.
- Filter for options with 1, 7, or 28 day expiry times.
- Add columns for time to expiry in seconds and years.
- Add columns for the mid price of calls and puts.

NOTE: I actually don't think I'm going to do these:
- Add a column for the 13 week treasury bill (`^IRX`) that we'll use as a risk-free interest rate in our calculations.
- Replace the given implied volatility values with our own calculation.

In [3]:
data_years = [2020, 2021, 2022, 2023]
data_dirs = [Path(str(year)) for year in data_years]

newcols = {
    " [UNDERLYING_LAST]": "underlying_last",
    " [EXPIRE_UNIX]": "expire_unix",
    "[QUOTE_UNIXTIME]": "quote_unix",
    " [QUOTE_READTIME]": "quote_time",
    " [STRIKE]": "strike",
    " [C_IV]": "call_iv",
    " [C_BID]": "call_bid",
    " [C_ASK]": "call_ask",
    " [C_LAST]": "call_last",
    " [P_IV]": "put_iv",
    " [P_BID]": "put_bid",
    " [P_ASK]": "put_ask",
    " [P_LAST]": "put_last",
    " [DTE]": "dte"
    }

# # Pull the data for ^IRX
# start_date = str(min(data_years)) + "-01-01"
# end_date = str(max(data_years)) + "-01-31"
# irx = yf.download("^IRX", start=start_date, end=end_date)

# display(irx)

# import matplotlib.pyplot as plt

# plt.plot(irx['Close'])



In [ ]:
output_dir = ".." # csv files will be saved to this directory at the end

dfs = []

for dir in data_dirs:

    dfs = []

    print(f"Beginning to process data in {dir}/")
    for csv_path in dir.glob("*.txt"):
        
        # rename the columns for convenience
        df = pd.read_csv(csv_path)
        df.rename(columns=newcols, inplace=True)
        df = df[newcols.values()]

        # filter for options with 1 DTE, 7 DTE, or 28 DTE
        df = df[(df["dte"].isin([1, 7, 28]))]

        # add a column for time to expiry in seconds
        df['tte_unix'] = df['expire_unix'] - df['quote_unix']

        # add a column for the time to expiry in years
        df['tte_years'] = df['tte_unix']/(60*60*24*365)

        # make sure the quote time column is a datetime
        df['quote_time'] = pd.to_datetime(df['quote_time'])

        # make sure numeric columns are numeric
        numeric_columns = ["call_iv", "call_bid", "call_ask", "call_last", "put_iv", "put_bid", "put_ask", "put_last"]
        for col in numeric_columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

        # add a mid price for the options
        df['call_bid'] = pd.to_numeric(df['call_bid'], errors='coerce')
        df['call_ask'] = pd.to_numeric(df['call_ask'], errors='coerce')
        df['put_bid'] = pd.to_numeric(df['put_bid'], errors='coerce')
        df['put_ask'] = pd.to_numeric(df['put_ask'], errors='coerce')

        for option_type in ['call', 'put']:
            df[f'{option_type}_mid'] = (df[f'{option_type}_ask'] + df[f'{option_type}_bid'])/2

        # # replace the given IV values with our own calculation
        # df['call_iv'] = df.apply(
        #     lambda row: implied_volatility_call(
        #         market_price=row['call_mid'],
        #         S0 = row['underlying_last'],
        #         K=row['strike'],
        #         t=row['tte_years'],
        #         r=0.0
        #     ),
        #     axis=1
        # )

        # df['put_iv'] = df.apply(
        #     lambda row: implied_volatility_put(
        #         market_price=row['put_mid'],
        #         S0 = row['underlying_last'],
        #         K=row['strike'],
        #         t=row['tte_years'],
        #         r=0.0
        #     ),
        #     axis=1
        # )



        dfs.append(df)
        print(f"Added {len(df)} rows from {csv_path}.")

    big_df = pd.concat(dfs, ignore_index=True)
    big_df.to_csv(f"{output_dir}/spx_options_{dir}.csv", index=False)
        



Beginning to process data in 2020/


/tmp/ipykernel_2920569/158595891.py:13: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


Added 8520 rows from 2020/spx_eod_202003.txt.


/tmp/ipykernel_2920569/158595891.py:13: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


Added 8437 rows from 2020/spx_eod_202004.txt.
Added 6467 rows from 2020/spx_eod_202001.txt.
Added 7631 rows from 2020/spx_eod_202006.txt.


...

Added 10088 rows from 2022/spx_eod_202209.txt.
Added 6516 rows from 2022/spx_eod_202203.txt.
Beginning to process data in 2023/
Added 8375 rows from 2023/spx_eod_202311.txt.


/tmp/ipykernel_2920569/158595891.py:13: DtypeWarning: Columns (8,9,10,11,12,15,17,18,20,21,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


Added 10156 rows from 2023/spx_eod_202308.txt.
Added 9165 rows from 2023/spx_eod_202312.txt.
Added 7873 rows from 2023/spx_eod_202310.txt.
Added 7633 rows from 2023/spx_eod_202301.txt.
Added 8905 rows from 2023/spx_eod_202306.txt.
Added 8838 rows from 2023/spx_eod_202309.txt.
Added 8914 rows from 2023/spx_eod_202307.txt.
Added 7382 rows from 2023/spx_eod_202304.txt.
Added 6665 rows from 2023/spx_eod_202302.txt.


/tmp/ipykernel_2920569/158595891.py:13: DtypeWarning: Columns (8,9,10,11,12,15,17,18,20,21,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


Added 8651 rows from 2023/spx_eod_202305.txt.
Added 7636 rows from 2023/spx_eod_202303.txt.


In [5]:
df = pd.read_csv(f"{output_dir}/spx_options_2020.csv")

In [6]:
display(df)

,underlying_last,expire_unix,quote_unix,quote_time,strike,call_iv,call_bid,call_ask,call_last,put_iv,put_bid,put_ask,put_last,dte,tte_unix,tte_years,call_mid,put_mid
0,3002.74,1583355600,1583269200,2020-03-03 16:00:00,1800.0,4.69707,1200.81,1210.10,1210.6,2.00098,0.00,0.05,0.10,1.0,86400,0.002740,1205.455,0.025
1,3002.74,1583355600,1583269200,2020-03-03 16:00:00,1900.0,NaN,1097.10,1107.30,1130.7,1.79853,0.00,0.05,0.05,1.0,86400,0.002740,1102.200,0.025
2,3002.74,1583355600,1583269200,2020-03-03 16:00:00,1950.0,3.83174,1050.80,1060.11,0.0,1.70212,0.00,0.05,0.00,1.0,86400,0.002740,1055.455,0.025
3,3002.74,1583355600,1583269200,2020-03-03 16:00:00,2000.0,NaN,995.10,1005.40,1102.6,1.60686,0.00,0.05,0.05,1.0,86400,0.002740,1000.250,0.025
4,3002.74,1583355600,1583269200,2020-03-03 16:00:00,2050.0,NaN,945.60,955.79,0.0,1.51524,0.00,0.05,0.05,1.0,86400,0.002740,950.695,0.025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82083,3624.23,1609189200,1606770000,2020-11-30 16:00:00,5000.0,0.34457,0.00,0.10,0.0,0.72737,1379.59,1389.40,0.00,28.0,2419200,0.076712,0.050,1384.495
82084,3624.23,1609189200,1606770000,2020-11-30 16:00:00,5100.0,0.37508,0.00,0.15,0.0,0.73548,1465.20,1490.51,0.00,28.0,2419200,0.076712,0.075,1477.855
82085,3624.23,1609189200,1606770000,2020-11-30 16:00:00,5200.0,0.36511,0.00,0.04,0.0,0.82286,1579.50,1589.30,0.00,28.0,2419200,0.076712,0.020,1584.400
82086,3624.23,1609189200,1606770000,2020-11-30 16:00:00,5300.0,0.38338,0.00,0.05,0.0,0.87009,1679.51,1689.30,0.00,28.0,2419200,0.076712,0.025,1684.405
